In [2]:
import dask.dataframe as ddf
import dask.distributed as dd
import pandas as pd
from progress.bar import Bar


In [23]:
def grid_to_lat_long(grid):
    # http://www.newsvhf.com/my-grid.html
    print(grid)

# assert grid_to_lat_long("CM87vl") == (37.4634447, -122.2256803)

In [24]:
spots_ddf = pd.read_csv(
    "data/reverse_beacon_network/20220101.csv",
    dtype={"db": "float64", "freq": "float64", "speed": "float64"},
)

681617

In [25]:
ak_indices = pd.read_csv(
    "/Users/dave/code/old-betsy/data/SWPC/AK_values/prepped/202201AK.csv",
    dtype={"geomagnetic_dipole_lat": "object", "geomagnetic_dipole_long": "object"},
)

In [26]:
# ak_indices = pd.read_csv("data/SWPC/AK_values/prepped/202201AK.csv").reset_index()
# spots_ddf = pd.read_csv("data/reverse_beacon_network/20220101.csv").reset_index()
spots_ddf["date"] = ddf.to_datetime(spots_ddf["date"])
ak_indices["start_time"] = ddf.to_datetime(ak_indices["start_time"])
ak_indices["end_time"] = ddf.to_datetime(ak_indices["end_time"])

len(spots_ddf)

681617

In [27]:
spots_ddf = spots_ddf[-spots_ddf["date"].isnull()]
len(spots_ddf)

681616

In [28]:
spots_pd = spots_ddf #pd.DataFrame(spots_ddf.compute())
ak_indices_pd = ak_indices # pd.DataFrame(ak_indices.compute())

len(spots_pd)

681616

In [29]:
with Bar("Processing", max=len(spots_pd), suffix="%(percent)d%%") as bar:
    for spot in spots_pd.itertuples(name="Spot"):
        ak_filtered = ak_indices_pd[ak_indices_pd["start_time"].values < spot.date]
        ak_filtered = ak_filtered[spot.date < ak_filtered["end_time"].values]

        for station in ak_indices["station"].unique():
            spots_pd.at[spot.Index, f"{station}_A_index"] = ak_filtered[
                ak_filtered["station"] == station
            ]["A_index"].values
            spots_pd.at[spot.Index, f"{station}_K_index"] = ak_filtered[
                ak_filtered["station"] == station
            ]["K_index"].values

        bar.next()
    bar.finish()


In [ ]:
spots_pd.to_csv('data/full.csv')  

In [3]:
prepped = pd.read_csv('data/full.csv')  

In [4]:
prepped

,Unnamed: 0,callsign,de_pfx,de_cont,freq,band,dx,dx_pfx,dx_cont,mode,...,P. Tunguska_A_index,P. Tunguska_K_index,Petropavlovsk_A_index,Petropavlovsk_K_index,Planetary(estimated Ap)_A_index,Planetary(estimated Ap)_K_index,Tiksi Bay_A_index,Tiksi Bay_K_index,Wingst_A_index,Wingst_K_index
0,0,MW0MUT,GW,EU,3521.0,80m,DL5NDH,DL,EU,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
1,1,DK0TE,DL,EU,3521.0,80m,DL5NDH,DL,EU,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
2,2,DK0TE,DL,EU,3518.0,80m,LB2TB,LA,EU,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
3,3,DR4W,DL,EU,3521.0,80m,DL5NDH,DL,EU,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
4,4,UA4M,UA,EU,3521.0,80m,DL5NDH,DL,EU,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681611,681611,DL0PF,DL,EU,3560.0,80m,EG4CIB,EA,EU,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,3.0,-1.0,-1.0,16.0,4.0
681612,681612,SA1CCQ,SM,EU,3550.0,80m,DO6IH,DL,EU,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,3.0,-1.0,-1.0,16.0,4.0
681613,681613,WC2L,K,NaN,7055.0,40m,UE3BXG,UA,EU,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,3.0,-1.0,-1.0,16.0,4.0
681614,681614,W2NAF,K,NaN,10120.0,30m,AE0EE,K,NaN,CQ,...,-1.0,-1.0,-1.0,-1.0,11.0,3.0,-1.0,-1.0,16.0,4.0


In [7]:
prepped.columns

Index(['Unnamed: 0', 'callsign', 'de_pfx', 'de_cont', 'freq', 'band', 'dx',
       'dx_pfx', 'dx_cont', 'mode', 'db', 'date', 'speed', 'tx_mode',
       'Beijing_A_index', 'Beijing_K_index', 'Belsk_A_index', 'Belsk_K_index',
       'Boulder_A_index', 'Boulder_K_index', 'Cape Chelyuskin_A_index',
       'Cape Chelyuskin_K_index', 'Chambon-la-foret_A_index',
       'Chambon-la-foret_K_index', 'College_A_index', 'College_K_index',
       'Dixon Island_A_index', 'Dixon Island_K_index',
       'Fredericksburg_A_index', 'Fredericksburg_K_index',
       'Gottingen(provisional Ap)_A_index',
       'Gottingen(provisional Ap)_K_index', 'Kergulen Island_A_index',
       'Kergulen Island_K_index', 'Krenkel_A_index', 'Krenkel_K_index',
       'Learmonth_A_index', 'Learmonth_K_index', 'St. Petersburg_A_index',
       'St. Petersburg_K_index', 'Magadan_A_index', 'Magadan_K_index',
       'Moscow_A_index', 'Moscow_K_index', 'Murmansk_A_index',
       'Murmansk_K_index', 'Novosibirsk_A_index', 'Novosib

In [12]:
grouped = prepped.groupby(["dx_pfx", "dx_cont", "de_pfx", "de_cont", "date", "band", "mode"]).mean()

In [13]:
grouped

Unnamed: 0  \
dx_pfx dx_cont de_pfx de_cont date                band mode                  
3B8    AF      DL     EU      2022-01-01 08:12:18 15m  CQ          92198.0   
                              2022-01-01 11:03:04 15m  CQ         220897.0   
                              2022-01-01 11:12:13 15m  CQ         230523.0   
               ET     AF      2022-01-01 11:11:36 15m  CQ         229769.0   
               HA     EU      2022-01-01 23:41:07 40m  CQ         676897.0   
...                                                                    ...   
ZS     AF      VU     AS      2022-01-01 16:49:48 20m  NCDXF B    471873.0   
                              2022-01-01 18:07:47 20m  NCDXF B    516162.0   
                              2022-01-01 18:28:48 20m  NCDXF B    527833.0   
                              2022-01-01 20:40:56 20m  NCDXF B    596659.0   
                              2022-01-01 21:59:01 17m  NCDXF B    634756.0   

                                                                   freq    db  \
dx_pfx dx_cont de_pfx de_cont date                band mode                     
3B8    AF      DL     EU      2022-01-01 08:12:18 15m  CQ       21030.0   6.0   
                              2022-01-01 11:03:04 15m  CQ       21022.6   5.0   
                              2022-01-01 11:12:13 15m  CQ       21022.6  10.0   
               ET     AF      2022-01-01 11:11:36 15m  CQ       21022.6  12.0   
               HA     EU      2022-01-01 23:41:07 40m  CQ        7013.0  17.0   
...                                                                 ...   ...   
ZS     AF      VU     AS      2022-01-01 16:49:48 20m  NCDXF B  14100.0   5.0   
                              2022-01-01 18:07:47 20m  NCDXF B  14100.0   6.0   
                              2022-01-01 18:28:48 20m  NCDXF B  14100.0   5.0   
                              2022-01-01 20:40:56 20m  NCDXF B  14100.0   3.0   
                              2022-01-01 21:59:01 17m  NCDXF B  18110.0   3.0   

                                                                speed  \
dx_pfx dx_cont de_pfx de_cont date                band mode             
3B8    AF      DL     EU      2022-01-01 08:12:18 15m  CQ        22.0   
                              2022-01-01 11:03:04 15m  CQ        22.0   
                              2022-01-01 11:12:13 15m  CQ        16.0   
               ET     AF      2022-01-01 11:11:36 15m  CQ        23.0   
               HA     EU      2022-01-01 23:41:07 40m  CQ        25.0   
...                                                               ...   
ZS     AF      VU     AS      2022-01-01 16:49:48 20m  NCDXF B   22.0   
                              2022-01-01 18:07:47 20m  NCDXF B   22.0   
                              2022-01-01 18:28:48 20m  NCDXF B   22.0   
                              2022-01-01 20:40:56 20m  NCDXF B   22.0   
                              2022-01-01 21:59:01 17m  NCDXF B   24.0   

                                                                Beijing_A_index  \
dx_pfx dx_cont de_pfx de_cont date                band mode                       
3B8    AF      DL     EU      2022-01-01 08:12:18 15m  CQ                  -1.0   
                              2022-01-01 11:03:04 15m  CQ                  -1.0   
                              2022-01-01 11:12:13 15m  CQ                  -1.0   
               ET     AF      2022-01-01 11:11:36 15m  CQ                  -1.0   
               HA     EU      2022-01-01 23:41:07 40m  CQ                  -1.0   
...                                                                         ...   
ZS     AF      VU     AS      2022-01-01 16:49:48 20m  NCDXF B             -1.0   
                              2022-01-01 18:07:47 20m  NCDXF B             -1.0   
                              2022-01-01 18:28:48 20m  NCDXF B             -1.0   
                              2022-01-01 20:40:56 20m  NCDXF B             -1.0   
                              2022-01-01 21:59:01

In [73]:
import untangle
import requests
import os

username = "KN6FMV"
password = "pke-adq@cwq5fua1XHG"

api_root = "https://www.hamqth.com/xml.php"

auth_uri = f"{api_root}?u={username}&p={password}"

document = untangle.parse(auth_uri)

session_id = document.HamQTH.session.session_id.cdata

data_dir = "data/hamqth"

callsign_to_lookup = "KN6FMV"
if os.path.exists(f"{data_dir}/{callsign_to_lookup}.xml"):
    callsign_result = untangle.parse(f"{data_dir}/{callsign_to_lookup}.xml")
    grid = callsign_result.HamQTH.search.grid.cdata
    print("Stored result: ", callsign_to_lookup, grid)
else:
    callsign_lookup_uri = f"{api_root}?id={session_id}&callsign={callsign_to_lookup}&prg=old-betsy"
    resp = requests.get(callsign_lookup_uri)

    if resp.status_code == 200:
        decoded_xml = resp.content.decode("ascii")
        os.makedirs(data_dir, exist_ok=True)
        with open(f"{data_dir}/{callsign_to_lookup}.xml", "w") as f:
            f.write(decoded_xml)
            f.close
        callsign_result = untangle.parse(decoded_xml)

        grid = callsign_result.HamQTH.search.grid.cdata

        print("Fresh result: ", callsign_to_lookup, grid)


Stored result:  KN6FMV CM87


'CM87'